In [ ]:
import pandas as pd
import numpy as np
#Data plotting
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn
from xgboost import XGBClassifier
import sklearn.metrics as metrics
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble  import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix


In [ ]:
#load the dataset
class DataUnderstanding:
    def __init__(self, data = None):
      self.df = data
    #Load Data
    def load_data(self,path):
      # Try reading with 'latin-1' encoding and specifying the delimiter as '\t'
      self.df = pd.read_csv(path, encoding='latin-1', delimiter='\t')
      return self.df
    def data_understanding(self):
      # First five rows of the dataset
      print('n\n\First five rows of the dataset')
      print('-' * 5)
      print(self.df.head())

      # Dataset Info
      print('n\n\Dataset Info')
      print('-' * 5)
      print(self.df.info())

      # Statistical Sumary of the dataset
      print('n\n\Stastical summary')
      print('-' * 5)
      print(self.df.describe())
# Total Null values per column
      print('\n\nNull values per column')
      print('-' * 5)
      print(self.df.isnull().sum())

      # Unique Values
      print("\n\nUNIQUE VALUES")
      print("-"*12)
      for col in self.df.columns:
          print(f"Column *{col}* has {self.df[col].nunique()} unique values")
          if self.df[col].nunique() < 12:
              print(f"Top unique values in the *{col}* include:")
              for idx in self.df[col].value_counts().index:
                  print(f"- {idx}")
          print("")

      # Total Duplicates in the dataset
      print('n\n\Total duplicated rows in the dataset')
      print('-'* 5)
      print(self.df.duplicated().sum())

In [ ]:
data_understanding = DataUnderstanding()
df = data_understanding.load_data('ma3route_crashes_algorithmcode.csv')
data_understanding.data_understanding()

n
\First five rows of the dataset
-----
  crash_id,crash_datetime,crash_date,latitude,longitude,n_crash_reports,contains_fatality_words,contains_pedestrian_words,contains_matatu_words,contains_motorcycle_words
0  1,06/06/2018 20:39,06/06/2018,-1.26302986,36.7...                                                                                                                     
1  2,17/08/2018 06:15,17/08/2018,-0.829710012,37....                                                                                                                     
2  3,25/05/2018 17:51,25/05/2018,-1.12530079,37.0...                                                                                                                     
3  4,25/05/2018 18:11,25/05/2018,-1.740957808,37....                                                                                                                     
4  5,25/05/2018 21:59,25/05/2018,-1.259392311,36....                                                          

In [ ]:
#data cleaning
#checking for missing values
df.isna().sum()

,0
"crash_id,crash_datetime,crash_date,latitude,longitude,n_crash_reports,contains_fatality_words,contains_pedestrian_words,contains_matatu_words,contains_motorcycle_words",0


Dataset has no missing vaklues

In [ ]:
#checking for duplictaes
df.duplicated().sum()

0

Dataset has no duplicated values

Uniformity

In [ ]:
#looking at the columns
df.columns

Index(['crash_id,crash_datetime,crash_date,latitude,longitude,n_crash_reports,contains_fatality_words,contains_pedestrian_words,contains_matatu_words,contains_motorcycle_words'], dtype='object')

In [ ]:
print(df.columns)

Index(['crash_id,crash_datetime,crash_date,latitude,longitude,n_crash_reports,contains_fatality_words,contains_pedestrian_words,contains_matatu_words,contains_motorcycle_words'], dtype='object')


In [ ]:
# Display cells with whitespace in specific columns
for col in df.select_dtypes(include=['object']).columns:
    print(f"Whitespace in column '{col}':")
    print(df[df[col].str.contains(r'^\s|\s$', na=False)][col])

Whitespace in column 'crash_id,crash_datetime,crash_date,latitude,longitude,n_crash_reports,contains_fatality_words,contains_pedestrian_words,contains_matatu_words,contains_motorcycle_words':
Series([], Name: crash_id,crash_datetime,crash_date,latitude,longitude,n_crash_reports,contains_fatality_words,contains_pedestrian_words,contains_matatu_words,contains_motorcycle_words, dtype: object)


In [ ]:
# Strip whitespace from column names
df.columns = df.columns.str.strip()

In [ ]:
# Proceed with your code after ensuring the column exists
latitude_out_of_range = df[(df['latitude'] < -1.5) | (df['latitude'] > 1.5)].shape[0]
longitude_out_of_range = df[(df['longitude'] < 36) | (df['longitude'] > 38)].shape[0]
print(f"\nEntries with latitude out of expected range: {latitude_out_of_range}")
print(f"Entries with longitude out of expected range: {longitude_out_of_range}")


KeyError: 'latitude'

In [ ]:
#EDA
# univariate analysis
